# [Articles sharing and reading from CI&T DeskDrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop)

Датасет "Articles sharing and reading from CI&T DeskDrop", включающий в себя собранные за один год логи DeskDrop — платформы для внутренних коммуникаций, разработанной CI&T и ориентированной на компании, использующие Google Workspace (Google G Suite). Среди прочего, эта платформа позволяет сотрудникам компаний делиться актуальными статьями со своими коллегами.

In [2]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

В датасете содержится около 73 тысяч записей о взаимодействии пользователей с более чем тремя тысячами публичных статей, размещённых на платформе.

Информация в наборе данных:
Оригинальный URL, название и текст статьи.
Контекст посещений пользователей, например дата/время, клиент (мобильное приложение/браузер) и геолокация.
Различные типы взаимодействия, что позволяет сделать вывод об уровне заинтересованности пользователя в статьях, например комментарии → лайки → просмотры.

Для временной метки существует два возможных типа событий:

- CONTENT SHARED — статья была опубликована на платформе и доступна для пользователей;
- CONTENT REMOVED — статья была удалена с платформы и недоступна для дальнейших рекомендаций.
Для простоты мы рассматриваем здесь только тип события CONTENT SHARED.

## Задание 6.1

Отфильтруйте данные так, чтобы остались только объекты с типом события CONTENT SHARED. Сколько таких объектов в получившейся таблице?

In [3]:
articles_df = pd.read_csv('data/shared_articles.zip')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head(5)


,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en


In [4]:
interactions_df = pd.read_csv('data/users_interactions.zip')
interactions_df.head(10)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN
5,1465413742,VIEW,310515487419366995,-8763398617720485024,1395789369402380392,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,MG,BR
6,1465415950,VIEW,-8864073373672512525,3609194402293569455,1143207167886864524,NaN,NaN,NaN
7,1465415066,VIEW,-1492913151930215984,4254153380739593270,8743229464706506141,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
8,1465413762,VIEW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
9,1465413771,VIEW,3064370296170038610,3609194402293569455,1143207167886864524,NaN,NaN,NaN


Давайте предварительно преобразуем столбцы personId, contentId в таблицах к строкам. Это преобразование пригодится нам в дальнейшем:

In [5]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

В колонке eventType описаны действия, которые могли совершать пользователи при взаимодействии со статьёй:

- VIEW — просмотр,
- LIKE — лайк,
- COMMENT CREATED — комментарий,
- FOLLOW — подписка,
- BOOKMARK — добавление в закладки.
  
В первую очередь нам необходимо понять, как определить, что какая-то статья популярнее других. Если бы из возможных реакций у нас были только лайки или только просмотры, то статьи было бы легко ранжировать в соответствии с этими значениями. Однако у нас есть информация о различных действиях пользователя, и на её основе мы должны создать некий универсальный индекс популярности. Составим его из реакций пользователей, придав им разные веса:

In [6]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type_strength[x])
interactions_df['eventStrength'].mean().round(2)

1.24

Ранее мы говорили, что рекомендательные системы подвержены проблеме холодного старта — в таких случаях создавать рекомендации намного сложнее

## Задание 6.3

Чтобы получить хоть какую-то информацию, на которую можно будет опираться, оставьте только тех пользователей, которые взаимодействовали хотя бы с пятью статьями. Сколько всего таких пользователей?

In [7]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .size()
    .groupby('personId').size()
)
print('# users: %d' % len(users_interactions_count_df))

users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


## Задание 6.4

Теперь оставим только те взаимодействия, которые касаются только отфильтрованных пользователей (то есть тех, которые взаимодействовали как минимум с пятью статьями). Сколько всего таких взаимодействий?

In [8]:
print(f'# of interactions: {len(interactions_df)}')
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'personId',
               right_on = 'personId')
print(f'# of interactions from users with at least 5 interactions: {len(interactions_from_selected_users_df)}')

# of interactions: 72312
# of interactions from users with at least 5 interactions: 69868


Сейчас каждое отдельное взаимодействие пользователя со статьёй выделено в отдельную запись, то есть пользователь мог просмотреть статью, лайкнуть и прокомментировать её, и всё это отразилось в трёх действиях. Давайте для удобства соединим все эти действия в некоторый коэффициент, который будет отражать интерес пользователя к статье. Так как каждому возможному действию мы ранее уже присвоили вес, то, по сути, нам нужно просто сложить все действия. Однако полученное число будет увеличиваться с количеством действий, и будет очень большой разброс возможных значений. В таких случаях обычно логарифмируют полученный результат с помощью следующей функции:

In [9]:
def smooth_user_preference(x):
    return math.log(1+x, 2)

## Задание 6.5

Примените упомянутое выше преобразование для логарифмирования к сумме весов для взаимодействия пользователя с каждой конкретной статьёй. Также сохраните для каждой пары «пользователь — статья» значение времени последнего взаимодействия.

Найдите среднее по признаку с получившимися временными отсечками. Округлите результат до двух знаков после точки-разделителя.

Так как наши данные отсортированы по дате, то для того, чтобы выбрать последнее взаимодействие, необходимо использовать метод max().

В этом задании необходимо работать с данными, полученными в задании 6.4.

In [10]:
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].max()
)
        
interactions_full_df = interactions_full_df.reset_index()

print('Mean of last_timestamp: ', interactions_full_df['last_timestamp'].mean().round(2))

print(f'# of unique user/item interactions: {len(interactions_full_df)}')
interactions_full_df.head(10)

Mean of last_timestamp:  1470605340.04
# of unique user/item interactions: 39106


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994342
5,-1007001694607905623,8729086959762650511,1.000000,1487240086
6,-1032019229384696495,-1006791494035379303,1.000000,1469129122
7,-1032019229384696495,-1039912738963181810,1.000000,1459376415
8,-1032019229384696495,-1081723567492738167,2.000000,1464054096
9,-1032019229384696495,-1111518890369033396,1.000000,1470109122


Разумеется, для того чтобы впоследствии оценить качество построенной рекомендательной системы, нам нужно разделить выборку на обучающую и тестовую. Так как в реальности рекомендации строятся на основе исторических данных о пользователе и контенте, сделаем в нашей задаче разбиение на обучающую и тестовую выборки по временной отсечке.

## Задание 6.6

Разделите данные на обучающую и тестовую выборки, выбрав в качестве временной отсечки значение 1475519545. Сколько объектов попало в обучающую выборку?

In [11]:
split_ts = 1475519545
interactions_train_df = interactions_full_df.query('last_timestamp < @split_ts').copy()
interactions_test_df = interactions_full_df.query('last_timestamp >= @split_ts').copy()

print(len(interactions_train_df))


29325


Для удобства дальнейшего измерения качества рекомендаций преобразуйте данные так, чтобы получить таблицу в формате, где строка соответствует пользователю, а столбцы будут истинными предпочтениями и рекомендациями в формате списков. На место пустых ячеек поместите пустые списки.

In [12]:
final_df = (
    interactions_train_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    interactions_test_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]
final_df.head()

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224]


## Задание 6.7

Осталось совсем немного — скоро вы получите свою первую систему рекомендаций! Мы будем строить popular-based-модель, а значит, нам необходимо найти самые популярные статьи.

Посчитайте популярность каждой статьи как сумму всех логарифмических «оценок» взаимодействий с ней (используя только обучающую выборку). Выберите ID самой популярной статьи:

In [13]:
popular = (
    interactions_train_df
    .groupby('contentId')
    .eventStrength.sum().reset_index()
    .sort_values('eventStrength', ascending=False)
    .contentId.values
)
popular[0]

'-6783772548752091658'

## Задание 6.8

Постройте систему рекомендаций. Оцените качество с помощью precision@10 для каждого пользователя (доля угаданных рекомендаций). После этого усредните результат по всем пользователям.

Для вычисления precision@10 воспользуйтесь следующей функцией:

```py
def precision(column):
    return (final_df.apply(lambda row:
            len(set(row['true_test']).intersection(set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),axis=1)).mean()
```
Итоговый результат округлите до трёх знаков после точки-разделителя.

В этом задании необходимо использовать данные, полученные в задании 6.6 (для которых приведён скриншот выше).

In [14]:
def precision(column):
    return (final_df.apply(lambda row:
            len(set(row['true_test']).intersection(set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),axis=1)).mean()

top_k = 10
 
final_df['popular'] = (
    final_df.true_train
    .apply(
        lambda x:
        popular[~np.in1d(popular, x)][:top_k]
    )
)
def calc_precision(column):
    return (
        final_df
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),
            axis=1)).mean()
calc_precision('popular')

0.006454207722621089

Качество получилось не очень высоким, но ведь и рекомендации у нас были неперсонализированными.

Примечание. Стоит отметить, что качество РС оценивается не так, как в задачах классификации: показателей выше 0.5 добиться практически невозможно, и даже результат 0.1–0.2 — индикатор высокого качества.

## Задание 7.1

В магазине три покупателя и три товара. Ниже приведены полученные векторы признаков для каждого пользователя и продукта. Основываясь на этих данных, выберите, какой продукт вы бы с наибольшей вероятностью рекомендовали пользователю № 2.

Векторы признаков для пользователей:

- (1.73, 0.01, 5.22)
- (0.03, 4.41, 2.05)
- (1.13, 0.89, 3.76)

Векторы характеристик для продуктов:

- (3.29, 3.44, 3.67)
- (0.82, 9.71, 3.88)
- (8.34, 1.72, 0.02)

In [19]:
import numpy as np

# Вектор признаков пользователя № 2
user_features = np.array([0.03, 4.41, 2.05])

# Векторы характеристик продуктов
product_features = np.array([
    [3.29, 3.44, 3.67],
    [0.82, 9.71, 3.88],
    [8.34, 1.72, 0.02]
])

# Вычисление косинусного расстояния между вектором признаков пользователя № 2 и каждым из векторов характеристик продуктов
cosine_similarities = [np.dot(user_features, product) / (np.linalg.norm(user_features) * np.linalg.norm(product)) for product in product_features]

# Нахождение индекса продукта с максимальным косинусным расстоянием (наиболее подходящего продукта для рекомендации)
recommended_product_index = np.argmax(cosine_similarities) + 1  # добавляем 1, так как индексы начинаются с 0

print("Наиболее подходящий продукт для рекомендации пользователю № 2:", recommended_product_index)

Наиболее подходящий продукт для рекомендации пользователю № 2: 2


## Задание 7.4

Вычислите метрику MRR (Mean Reciprocal Rank), если у нас есть следующие результаты рекомендаций (красным цветом отмечены неверные, зелёным — верные, ранжирование — слева направо). Округлите до двух знаков после точки-разделителя.
Красные (+) Зеленые (-)
- +----
- ---+-
- --+--

MRR (Mean Reciprocal Rank) вычисляется как среднее значение обратных рангов релевантных элементов. Для каждого запроса мы находим ранг первого верного рекомендованного элемента и затем находим обратный этого ранга.

Давайте вычислим MRR для представленных результатов:
- Для первого запроса верный элемент находится на первом месте. Обратный ранг: 1/1 = 1.
- Для второго запроса верный элемент находится на четвертом месте. Обратный ранг: 1/4.
- Для третьего запроса верный элемент находится на третьем месте. Обратный ранг: 1/3.
  
Теперь вычислим среднее значение обратных рангов:

MRR = (1/1 + 1/4 + 1/3) / 3 = (1 + 0.25 + 0.33) / 3 ≈ 0.53

Итак, MRR ≈ 0.53.